# Probablistic Matrix Factorization build 

Reference : https://www.kaggle.com/code/robottums/probabalistic-matrix-factorization-with-suprise 

The original code used surprise dataset, but my code reads numpy and dataframe 

In [136]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import random
import argparse
import pickle
import torch

In [137]:
# Example R matrix 
# the row means user and column means item 
# for example user0 rated item0 rating 1 and item4 rating 3. 
R = np.array([
    [1, 0, 0, 1, 3],
    [2, 0, 3, 1, 1],
    [1, 2, 0, 5, 0],
    [1, 0, 0, 4, 4],
    [2, 1, 5, 4, 0],
    [5, 1, 5, 4, 0],
    [0, 0, 0, 1, 0],
])

# The latent matrix P and Q will be the size of (7, d) and (d, 5)

In [138]:
class ProbabilisticMatrixFactorization(torch.nn.Module): 
    def __init__(self, R, d=10, lr=0.01, reg_param=0.01, epochs=10):
      super(ProbabilisticMatrixFactorization, self).__init__()
      self.R = R
      self.d = d
      self.lr = lr
      self.reg_param = reg_param 
      self.epochs = epochs 
      self.nusers = R.shape[0] 
      self.mitems = R.shape[1]

      # latent vectors
      self.user_vecs = np.random.normal(scale = 1. / self.d, size=(self.nusers, self.d)) 
      self.item_vecs = np.random.normal(scale = 1. / self.d, size=(self.mitems, self.d))

      """
      # biases 
      self.user_bias = np.zeros(self.nusers)
      self.item_bias = np.zeros(self.mitems)
      """

      self.all_mean = np.sum(self.R) / len(np.where(self.R > 0)[0])
      print("Average of all ratings:", self.all_mean)

    def rui(self):
      return self.rui

    def uv(self):
      return self.user_vecs

    def fit(self) :
      xi, yi = self.R.nonzero() 
      print("Model epochs:", self.epochs)
      for epoch in range(self.epochs):
        for i, j in zip(xi, yi):
          # i = user x j item y R[i, j] = r_ui 
          residual = self.R[i, j] - np.dot(self.user_vecs[i], self.item_vecs[j])
          if i == 0 and j == 3 and epoch % 10 == 0:
            print("[Epoch %2d] Target Score %d, Predicted Score %.4f, Residual %.4f" %(epoch, self.R[i, j], np.dot(self.user_vecs[i], self.item_vecs[j]), residual))
          temp = self.user_vecs[i, :] 
          self.user_vecs[i, :] += self.lr * residual * self.item_vecs[j, :] 
          self.item_vecs[j, :] += self.lr * residual * temp 


    def test(self, user_ind, item_ind):
      if user_ind >= self.nusers or item_ind >= self.mitems or user_ind < 0 or item_ind < 0:
        return self.all_mean
      nanCheck = np.dot(self.user_vecs[user_ind, :], self.item_vecs[item_ind, :]) 
      if np.isnan(nanCheck):
        return self.all_mean 
      else: 
        return np.dot(self.user_vecs[user_ind, :], self.item_vecs[item_ind, :]) 
      

In [139]:
# The loss of a single element is decreasing
factorizer = ProbabilisticMatrixFactorization(R, d=10, lr=0.01, reg_param=0.01, epochs=100)
factorizer.fit()

Average of all ratings: 2.590909090909091
Model epochs: 100
[Epoch  0] Target Score 1, Predicted Score 0.0026, Residual 0.9974
[Epoch 10] Target Score 1, Predicted Score 0.0812, Residual 0.9188
[Epoch 20] Target Score 1, Predicted Score 0.5861, Residual 0.4139
[Epoch 30] Target Score 1, Predicted Score 1.2536, Residual -0.2536
[Epoch 40] Target Score 1, Predicted Score 1.4917, Residual -0.4917
[Epoch 50] Target Score 1, Predicted Score 1.5166, Residual -0.5166
[Epoch 60] Target Score 1, Predicted Score 1.4514, Residual -0.4514
[Epoch 70] Target Score 1, Predicted Score 1.3613, Residual -0.3613
[Epoch 80] Target Score 1, Predicted Score 1.2773, Residual -0.2773
[Epoch 90] Target Score 1, Predicted Score 1.2094, Residual -0.2094


In [140]:
# Truth
R[0, 4]

3

In [141]:
# Predicted. (2.9 and 3 is similar)
factorizer.test(0,4)

2.820135821446655

# Using dataFrame

Here, we will have a dataframe not a matrix. 
The first column is user_id and second column is movie_id. The last column will be each user's rating about an item 

In [142]:
xi, yi = R.nonzero()

In [143]:
make_np_lst = []
for x, y in zip(xi, yi):
  make_np_lst.append([x, y, R[x,y]])
data = pd.DataFrame(make_np_lst, columns=['User_ID', 'Movie_ID', 'Rating'])
data.head()

,User_ID,Movie_ID,Rating
0,0,0,1
1,0,3,1
2,0,4,3
3,1,0,2
4,1,2,3


In [144]:
# The mean value of Rating 2.59 means average of all rating
data[data.columns[0]]
data.describe()

,User_ID,Movie_ID,Rating
count,22.000000,22.000000,22.000000
mean,2.772727,1.909091,2.590909
std,1.875451,1.477098,1.623022
min,0.000000,0.000000,1.000000
25%,1.000000,0.250000,1.000000
50%,3.000000,2.000000,2.000000
75%,4.000000,3.000000,4.000000
max,6.000000,4.000000,5.000000


In [145]:
# The code is slightly different from above code, because it reads from pandas dataframe.

class ProbabilisticMatrixFactorization(torch.nn.Module): 
    def __init__(self, dataframe, d=10, lr=0.01, reg_param=0.01, epochs=10):
      super(ProbabilisticMatrixFactorization, self).__init__()
      self.data = dataframe
      self.d = d
      self.lr = lr
      self.reg_param = reg_param 
      self.epochs = epochs 
      self.nusers = len(np.unique(self.data[self.data.columns[0]])) #R.shape[0] 
      self.mitems = len(np.unique(self.data[self.data.columns[1]])) #R.shape[1]
      print(self.nusers, self.mitems)

      # latent vectors
      self.user_vecs = np.random.normal(scale = 1. / self.d, size=(self.nusers, self.d)) 
      self.item_vecs = np.random.normal(scale = 1. / self.d, size=(self.mitems, self.d))

      """
      # biases 
      self.user_bias = np.zeros(self.nusers)
      self.item_bias = np.zeros(self.mitems)
      """

      self.all_mean = np.sum(self.data.iloc[:,2]) / self.data.shape[0]
      print("Average of all ratings:", self.all_mean)

    def rui(self):
      return self.rui

    def uv(self):
      return self.user_vecs

    def fit(self) :
      print("Model epochs:", self.epochs)
      for epoch in range(self.epochs):
        for ind, row in self.data.iterrows():
          i, j, r_ui = row
          residual = r_ui - np.dot(self.user_vecs[i], self.item_vecs[j])
          if ind == 3 and epoch % 10 == 0:
            print("[Epoch %2d] Target Score %d, Predicted Score %.4f, Residual %.4f" %(epoch, r_ui, np.dot(self.user_vecs[i], self.item_vecs[j]), residual))
          temp = self.user_vecs[i, :] 
          self.user_vecs[i, :] += self.lr * residual * self.item_vecs[j, :] 
          self.item_vecs[j, :] += self.lr * residual * temp 


    def test(self, user_ind, item_ind):
      if user_ind >= self.nusers or item_ind >= self.mitems or user_ind < 0 or item_ind < 0:
        return self.all_mean
      nanCheck = np.dot(self.user_vecs[user_ind, :], self.item_vecs[item_ind, :]) 
      if np.isnan(nanCheck):
        return self.all_mean 
      else: 
        return np.dot(self.user_vecs[user_ind, :], self.item_vecs[item_ind, :]) 
      

In [146]:
factorizer = ProbabilisticMatrixFactorization(data, d=10, lr=0.01, reg_param=0.01, epochs=200)
factorizer.fit()

7 5
Average of all ratings: 2.590909090909091
Model epochs: 200
[Epoch  0] Target Score 2, Predicted Score 0.0398, Residual 1.9602
[Epoch 10] Target Score 2, Predicted Score 0.1337, Residual 1.8663
[Epoch 20] Target Score 2, Predicted Score 0.5961, Residual 1.4039
[Epoch 30] Target Score 2, Predicted Score 1.4064, Residual 0.5936
[Epoch 40] Target Score 2, Predicted Score 1.6037, Residual 0.3963
[Epoch 50] Target Score 2, Predicted Score 1.6912, Residual 0.3088
[Epoch 60] Target Score 2, Predicted Score 1.8332, Residual 0.1668
[Epoch 70] Target Score 2, Predicted Score 1.9851, Residual 0.0149
[Epoch 80] Target Score 2, Predicted Score 2.1011, Residual -0.1011
[Epoch 90] Target Score 2, Predicted Score 2.1686, Residual -0.1686
[Epoch 100] Target Score 2, Predicted Score 2.1966, Residual -0.1966
[Epoch 110] Target Score 2, Predicted Score 2.1994, Residual -0.1994
[Epoch 120] Target Score 2, Predicted Score 2.1882, Residual -0.1882
[Epoch 130] Target Score 2, Predicted Score 2.1703, Resid

In [147]:
# Truth 
data[(data['User_ID'] == 0) & (data['Movie_ID'] == 3)]

,User_ID,Movie_ID,Rating
1,0,3,1


In [148]:
# Predicted. Rating and predicted value is somewhat similar 
user, item = 0, 3
print((data[(data['User_ID'] == user) & (data['Movie_ID'] == item)]))
factorizer.test(user, item)

   User_ID  Movie_ID  Rating
1        0         3       1


1.0544785004366863

In [149]:
# Rating and predicted value is very similar 
user, item = 2, 3
print((data[(data['User_ID'] == user) & (data['Movie_ID'] == item)]))
factorizer.test(user, item)

   User_ID  Movie_ID  Rating
9        2         3       5


5.025797307621225

In [150]:
# Let's look at all the predicted value with the truth
result = []
for ind, row in data.iterrows():
  user, item = row['User_ID'], row['Movie_ID']
  predicted_value = factorizer.test(user, item) 
  result.append([user, item, row['Rating'], predicted_value])
result = pd.DataFrame(result, columns=['User_ID', 'Movie_ID', 'Rating', 'Predicted'])
result

,User_ID,Movie_ID,Rating,Predicted
0,0,0,1,1.047712
1,0,3,1,1.054479
2,0,4,3,2.938521
3,1,0,2,2.059908
4,1,2,3,2.935280
5,1,3,1,1.062489
6,1,4,1,0.970934
7,2,0,1,0.990595
8,2,1,2,1.950477
9,2,3,5,5.025797


In [151]:
# Let's look at unknown rating that the model predicted. 
grid = []
for user_ind in range(R.shape[0]):
  user_rating = []
  for item_ind in range(R.shape[1]):
    predicted_value = factorizer.test(user_ind, item_ind)
    user_rating.append(int(predicted_value))
  grid.append(user_rating)
grid

[[1, 0, 0, 1, 2],
 [2, 0, 2, 1, 0],
 [0, 1, 3, 5, 4],
 [0, 1, 2, 3, 4],
 [2, 1, 4, 4, 2],
 [4, 0, 5, 3, 5],
 [0, 0, 0, 1, 1]]

In [152]:
# The original R matrix
R

array([[1, 0, 0, 1, 3],
       [2, 0, 3, 1, 1],
       [1, 2, 0, 5, 0],
       [1, 0, 0, 4, 4],
       [2, 1, 5, 4, 0],
       [5, 1, 5, 4, 0],
       [0, 0, 0, 1, 0]])